# Smart Agriculture IoT Simulation - Interactive Demo

This notebook provides an interactive demonstration of the Smart Agriculture IoT system featuring:
- IoT sensor simulation
- AI-powered crop health prediction
- Automated alert system
- Comprehensive visualization dashboard

## Contents
1. Setup and Configuration
2. Sensor Data Generation
3. AI Model Training
4. Real-Time Predictions and Alerts
5. Visualization Dashboard
6. Scenario Comparison
7. Management Recommendations

## 1. Setup and Configuration

In [ ]:
# Standard imports
import sys
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta

# Local modules
sys.path.append('./src')
import config
from iot_sensors import SensorNetwork
from data_generator import AgricultureDataGenerator
from ai_predictor import CropHealthPredictor, YieldPredictor
from alert_system import AlertSystem, IrrigationDecisionSystem, FertilizerDecisionSystem
from visualization import AgricultureDashboard

# Matplotlib settings
plt.style.use('seaborn-v0_8-darkgrid')
%matplotlib inline

print("✓ All modules loaded successfully")
print(f"\nProject: Smart Agriculture IoT")
print(f"Crop: {config.CROP_TYPE.title()}")
print(f"Field Area: {config.FIELD_AREA_HA} ha")

In [ ]:
# Create necessary directories
config.create_directories()
config.print_config()

## 2. Sensor Data Generation

Generate synthetic sensor data with realistic physics-based models.

In [ ]:
# Create data generator for optimal scenario
scenario = 'optimal'  # Options: 'optimal', 'drought', 'nutrient_deficiency', 'random'
simulation_days = 90  # 3 months for quick demo

generator = AgricultureDataGenerator(scenario=scenario, random_state=42)
sensor_data = generator.generate(days=simulation_days, save_to_file=False)

print(f"\n✓ Generated {len(sensor_data)} sensor readings")
print(f"\nData shape: {sensor_data.shape}")
print(f"\nFirst few records:")
sensor_data.head(10)

In [ ]:
# Display data summary
summary_stats = generator.get_summary_statistics(sensor_data)

print("Sensor Statistics:")
print("=" * 60)
for sensor, stats in summary_stats['sensor_stats'].items():
    print(f"{sensor:15s}: Mean={stats['mean']:7.2f}, Std={stats['std']:6.2f}, "
          f"Range=[{stats['min']:6.2f}, {stats['max']:6.2f}]")

print(f"\nStress Days:")
for stress_type, days in summary_stats['stress_days'].items():
    print(f"  {stress_type:15s}: {days} days")

print(f"\nManagement Actions:")
mgmt = summary_stats['management_actions']
print(f"  Irrigation: {mgmt['irrigation_events']} events ({mgmt['total_irrigation_mm']:.0f} mm total)")
print(f"  Fertilizer: {mgmt['fertilizer_events']} applications")

## 3. AI Model Training

Train crop health classifier and yield predictor.

In [ ]:
# Train Crop Health Classifier
print("Training Crop Health Prediction Model...")
print("=" * 60)

health_predictor = CropHealthPredictor(random_state=42)
health_metrics = health_predictor.train(sensor_data, save_model=False)

In [ ]:
# For yield prediction, generate multiple seasons
print("\nGenerating multi-season data for yield prediction...")
print("=" * 60)

# Create 5 seasons of data
seasons = []
for i in range(5):
    gen_temp = AgricultureDataGenerator(scenario='random', random_state=42+i)
    df_season = gen_temp.generate(days=180, save_to_file=False)
    df_season['season'] = i + 1
    seasons.append(df_season)

multi_season_data = pd.concat(seasons, ignore_index=True)
print(f"✓ Generated {len(multi_season_data)} records across 5 seasons")

# Train yield predictor
print("\nTraining Yield Prediction Model...")
print("=" * 60)

yield_predictor = YieldPredictor(random_state=42)
yield_metrics = yield_predictor.train(multi_season_data, save_model=False)

## 4. Real-Time Predictions and Alerts

Run predictions and check for alerts on the sensor data.

In [ ]:
# Initialize alert system
alert_system = AlertSystem()
predictions_list = []
all_alerts = []

print("Running predictions and checking alerts...")
from tqdm.notebook import tqdm

for idx, row in tqdm(sensor_data.iterrows(), total=len(sensor_data)):
    # Predict crop health
    features = {
        'soil_moisture': row['soil_moisture'],
        'temperature': row['temperature'],
        'humidity': row['humidity'],
        'light': row['light'],
        'ec': row['ec'],
        'days_since_planting': row['days_since_planting'],
        'rainfall_7d': row.get('rainfall_7d', 0)
    }
    
    health_result = health_predictor.predict(features)
    
    # Check alerts
    alerts = alert_system.check_conditions(row, timestamp=row['timestamp'])
    all_alerts.extend(alerts)
    
    # Store prediction
    predictions_list.append({
        'timestamp': row['timestamp'],
        'health_class': health_result['health_class'],
        'health_label': health_result['health_label'],
        'confidence': health_result['confidence']
    })

predictions_df = pd.DataFrame(predictions_list)

print(f"\n✓ Processed {len(predictions_df)} predictions")
print(f"✓ Generated {len(all_alerts)} alerts")

In [ ]:
# Display alert summary
alert_summary = alert_system.get_alert_summary()

print("\nAlert Summary:")
print("=" * 60)
print(f"Total Alerts: {alert_summary['total']}")

if alert_summary['total'] > 0:
    print(f"\nBy Type:")
    for alert_type, count in alert_summary['by_type'].items():
        print(f"  {alert_type:25s}: {count:3d}")
    
    print(f"\nBy Severity:")
    for severity, count in alert_summary['by_severity'].items():
        print(f"  {severity:25s}: {count:3d}")
    
    # Show some example alerts
    print(f"\nExample Alerts:")
    for alert in all_alerts[:5]:
        print(f"  [{alert.severity.upper()}] {alert.timestamp.strftime('%Y-%m-%d %H:%M')} - {alert.message}")

In [ ]:
# Health distribution
print("\nCrop Health Distribution:")
print("=" * 60)
health_counts = predictions_df['health_label'].value_counts()
for label, count in health_counts.items():
    percentage = (count / len(predictions_df)) * 100
    print(f"  {label:15s}: {count:5d} ({percentage:5.1f}%)")

avg_health = predictions_df['health_class'].mean()
print(f"\nAverage Health Score: {avg_health:.2f}/4.0")

## 5. Visualization Dashboard

Create comprehensive visualizations of the simulation results.

In [ ]:
# Create dashboard
dashboard = AgricultureDashboard(
    sensor_data=sensor_data,
    predictions=predictions_df,
    alerts=all_alerts
)

In [ ]:
# Plot sensor timelines
dashboard.plot_sensor_timeline(
    sensors=['soil_moisture', 'temperature', 'humidity', 'ec'],
    days=simulation_days
)

In [ ]:
# Plot sensor correlations
dashboard.plot_sensor_correlation()

In [ ]:
# Plot crop health timeline
dashboard.plot_crop_health_timeline()

In [ ]:
# Plot alert summary
if len(all_alerts) > 0:
    dashboard.plot_alert_summary()
else:
    print("No alerts to visualize")

In [ ]:
# Plot comprehensive daily summary
dashboard.plot_daily_summary()

## 6. Scenario Comparison

Compare different scenarios side-by-side.

In [ ]:
# Generate data for different scenarios
scenarios_to_compare = ['optimal', 'drought', 'nutrient_deficiency']
scenario_results = {}

for scenario_name in scenarios_to_compare:
    print(f"\nGenerating {scenario_name} scenario...")
    gen = AgricultureDataGenerator(scenario=scenario_name, random_state=42)
    df = gen.generate(days=90, save_to_file=False)
    
    # Calculate summary metrics
    scenario_results[scenario_name] = {
        'avg_moisture': df['soil_moisture'].mean(),
        'avg_temp': df['temperature'].mean(),
        'avg_ec': df['ec'].mean(),
        'total_rain': df['rainfall'].sum(),
        'drought_days': df['drought_stress'].sum(),
        'nutrient_days': df['nutrient_stress'].sum()
    }

print("\n✓ Scenario comparison complete")

In [ ]:
# Visualize scenario comparison
comparison_df = pd.DataFrame(scenario_results).T

fig, axes = plt.subplots(2, 3, figsize=(15, 8))
axes = axes.flatten()

metrics = ['avg_moisture', 'avg_temp', 'avg_ec', 'total_rain', 'drought_days', 'nutrient_days']
titles = ['Avg Soil Moisture (%)', 'Avg Temperature (°C)', 'Avg EC (dS/m)',
         'Total Rainfall (mm)', 'Drought Stress Days', 'Nutrient Stress Days']

for idx, (metric, title) in enumerate(zip(metrics, titles)):
    ax = axes[idx]
    comparison_df[metric].plot(kind='bar', ax=ax, color=['green', 'red', 'orange'])
    ax.set_title(title, fontweight='bold')
    ax.set_xlabel('')
    ax.grid(True, alpha=0.3, axis='y')

plt.tight_layout()
plt.suptitle('Scenario Comparison', fontsize=16, fontweight='bold', y=1.02)
plt.show()

print("\nScenario Comparison Table:")
print(comparison_df.round(2))

## 7. Management Recommendations

Get irrigation and fertilizer recommendations based on current conditions.

In [ ]:
# Initialize decision systems
irrigation_system = IrrigationDecisionSystem()
fertilizer_system = FertilizerDecisionSystem()

# Test on recent sensor data
recent_data = sensor_data.iloc[-1]

print("Current Sensor Readings:")
print("=" * 60)
print(f"  Soil Moisture: {recent_data['soil_moisture']:.1f}%")
print(f"  Temperature: {recent_data['temperature']:.1f}°C")
print(f"  Humidity: {recent_data['humidity']:.1f}%")
print(f"  EC: {recent_data['ec']:.2f} dS/m")
print(f"  Days since planting: {recent_data['days_since_planting']}")

# Irrigation recommendation
irrigation_rec = irrigation_system.recommend(recent_data)
print(f"\nIrrigation Recommendation:")
print("=" * 60)
print(f"  Action: {irrigation_rec['action']}")
print(f"  Amount: {irrigation_rec['amount_mm']} mm")
print(f"  Priority: {irrigation_rec['priority']}")
print(f"  Reason: {irrigation_rec['reason']}")

# Fertilizer recommendation
fertilizer_rec = fertilizer_system.recommend(recent_data, recent_data['days_since_planting'])
print(f"\nFertilizer Recommendation:")
print("=" * 60)
print(f"  Action: {fertilizer_rec['action']}")
print(f"  Amount: {fertilizer_rec['amount']}")
print(f"  Priority: {fertilizer_rec['priority']}")
print(f"  Reason: {fertilizer_rec['reason']}")

## Summary

This interactive demo demonstrated:
1. ✓ Realistic IoT sensor simulation with physics-based models
2. ✓ AI-powered crop health classification (Random Forest)
3. ✓ Yield prediction using Gradient Boosting
4. ✓ Automated alert system with multiple condition checks
5. ✓ Comprehensive visualization dashboard
6. ✓ Scenario comparison (optimal vs. drought vs. nutrient deficiency)
7. ✓ Automated irrigation and fertilizer recommendations

### Next Steps
- Run the full simulation: `python agriculture_simulation.py --scenario optimal --days 365`
- Explore different scenarios
- Customize sensor thresholds in `src/config.py`
- Integrate with real IoT hardware via MQTT
- Deploy as web dashboard using Dash or Streamlit
